To Do:  
* move video rotation.
* Legend.
* Check dates competibilities color to mesh.
* calc clim

test the basic functions needed to create the cube rendering

In [1]:
import pyvista as pv
import numpy as np
import netCDF4 as nc
import vtk
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib import colors
from skimage import io
import cv2
import logging
import pandas as pd
from scipy.interpolate import griddata
from scipy.spatial import QhullError
import os
from datetime import datetime, timedelta
import dateutil.parser
#import gemgis as gg

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Parameters

In [2]:
bg_padding = 400
cube_size = 10
cmap = 'seismic' #'RdBu'# 'PiYG' 'seismic_r'
clim = (-100,100)
clim = (0,2.6)
crop_coordinates = None # (40,55,0,20)

n_timepoints=100
frame_rate=30
event_csv_path='events.csv'

path_input_mesh = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
#path_input_color = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
path_input_color = '../../UFZ_RemoteSensing/NOAA-LAI-Europe-mon-2001-2012(1).nc'

output_path="output_cube10_seismic_bgPad400.mp4"



In [3]:
bg_padding = 220
cube_size = 20
cmap = 'PiYG' #'RdBu'# 'PiYG' 'seismic_r'
clim = (-100,100)
clim = (0,2.6)
lat_range = (40,59)
long_range = (0,21)
n_timepoints=100
frame_rate=60
event_csv_path=None#'events.csv'

is_interp = 'spatial'

path_input_mesh = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
#path_input_color = '../../UFZ_RemoteSensing/HOLAPS-H-JJA_anomaly-d-2001-2005.nc'
path_input_color = '../../UFZ_RemoteSensing/NOAA-LAI-Europe-mon-2001-2012(1).nc'

output_path="output_cube20_PiYG_bgPad200_cropped.mp4"

In [4]:
path_input_BG = '../backgound_map_tests/bluemarble.png'
n_interp = 10

## Functions

### Load, crop & resize data

In [5]:
def load_nc(path_input):
    ds = nc.Dataset(path_input)
    arr_name = [n for n in ds.variables.keys() if ds[n].ndim==3][0]
    arr_ref = ds[arr_name]
    
    logging.info(f'Read netCDF {arr_name} data. Shape={arr_ref.shape}')
    
    return ds, arr_ref

In [6]:
def format_time_arr_yearmonthday(ds):
    
    date_arr = ds['time'][:].astype(int)
    
    first_val = date_arr[0]
    
    if len(first_val.astype(str))!=8 or first_val.astype(str)[0] not in ['1','2']:

        try:
            start_date_obj = dateutil.parser.parse(ds.time_coverage_start)
            end_date_obj = dateutil.parser.parse(ds.time_coverage_end)
        except:
            raise Exception("Can't parse netCDF dataset time format")
            
        date_arr = [start_date_obj+timedelta(days=int(it-first_val)) for it in date_arr]
        date_arr = np.array([int(it.strftime("%Y%m%d")) for it in date_arr])
            
    return date_arr

In [7]:
def format_date_for_legend(date):
    date_obj = datetime.strptime(date, '%Y%m%d')
    return date_obj.strftime("%b %d, %Y")

In [8]:
def get_days_delta(date1_int, date2_int):
    return (datetime.strptime(str(date1_int), '%Y%m%d') - datetime.strptime(str(date2_int), '%Y%m%d')).days

In [9]:
def map_mesh_dates_to_color_idx(dates_mesh, dates_color):
    closest_smaller_idx_in_color = [np.where((dates_color-d)<=0)[0][-1] for d in dates_mesh]
    distance_in_days_from_smaller = np.array([get_days_delta(d, dates_color[closest_smaller_idx_in_color[i]]) for i,d in enumerate(dates_mesh)])
    distance_in_days_from_larger = np.array([get_days_delta(dates_color[closest_smaller_idx_in_color[i]+1], d) 
                                             if dates_color.size>closest_smaller_idx_in_color[i]+1 else np.inf
                                             for i,d in enumerate(dates_mesh)])
    
    relative_distance_from_smaller = distance_in_days_from_smaller / (distance_in_days_from_smaller+distance_in_days_from_larger)
    
    idxs_as_float = np.array(closest_smaller_idx_in_color) + relative_distance_from_smaller
    
    return idxs_as_float

In [10]:
def interpolate_between_slices(arr, arr_next, weight_next):
    interp_arr = arr*(1-weight_next)+arr_next*weight_next
    return interp_arr

In [11]:
def load_and_resize_arr(arr_ref, i_date, ds, is_min_zero, 
                        lat_min_max, long_min_max, 
                        data_shape, cube_size, is_interp=False):
    
    arr = load_arr_2D(arr_ref, int(i_date), is_min_zero)
    
    if not float(i_date).is_integer():
        arr_next = load_arr_2D(arr_ref, int(i_date+1), is_min_zero)
        arr = interpolate_between_slices(arr, arr_next, i_date%1)
    
    arr = get_world_part(arr, lat_min_max, long_min_max, ds)
    arr = cv2.resize(arr, (data_shape[1]//cube_size, data_shape[0]//cube_size), 
                                     interpolation=cv2.INTER_NEAREST).T
    
    if is_interp:
        arr = interpolate_nans(arr)
    
    return arr

In [12]:
def load_arr_2D(arr, arr_slice=0, min_zero=False, fill_val=-9999):
    arr = arr[arr_slice]
    arr = arr.filled(fill_value=np.nan)
    arr = arr - np.nanmin(arr) if min_zero else arr
    
    return arr

In [13]:
def find_min_max(coor_arr_mesh, coor_arr_color, user_range):
    
    if user_range is None:
        user_range = [np.nan, np.nan]
        
    coor_min = np.nanmax((np.nanmin(coor_arr_mesh), np.nanmin(coor_arr_color), user_range[0]))
    coor_max = np.nanmin((np.nanmax(coor_arr_mesh), np.nanmax(coor_arr_color), user_range[1]))
    
    return coor_min, coor_max

In [14]:
def find_idx_nearest(arr, val):
    return (np.abs(arr - val)).argmin()

def get_world_part(arr, lat_arr_src, long_arr_src, ds=None, padding=0):
    
    if ds is None:
        lat_arr_dst = np.linspace(-90, 90, arr.shape[0])
        long_arr_dst = np.linspace(-180, 180, arr.shape[1])
    else:
        lat_arr_dst = ds["latitude"][:]
        long_arr_dst = ds["longitude"][:]

    min_lat_idx = find_idx_nearest(lat_arr_dst, np.min(lat_arr_src))
    max_lat_idx = find_idx_nearest(lat_arr_dst, np.max(lat_arr_src))

    min_long_idx = find_idx_nearest(long_arr_dst, np.min(long_arr_src))
    max_long_idx = find_idx_nearest(long_arr_dst, np.max(long_arr_src))

    arr = arr[min_lat_idx-padding:max_lat_idx+padding, min_long_idx-padding:max_long_idx+padding]
    
    return arr

In [15]:
def make_BG_mesh(path_input_BG, bg_padding, lat_min, lat_max, long_min, long_max):
    bg = np.flip(io.imread(path_input_BG), axis=0)
    bg = get_world_part(bg, (lat_min, lat_max), (long_min, long_max), padding=bg_padding)
    data_shape = [s-bg_padding*2 for s in bg.shape[:2]]
    path_bg = 'tmp.png'
    io.imsave(path_bg, np.flip(bg,axis=0))
    bg = pv.read(path_bg)
    bg.origin = (-bg_padding, -bg_padding, 0)
    
    return bg, data_shape

In [16]:
def save_nc_with_interp(arr_interp, ds, path_color_interp):
    
    var_names = list(ds.variables.keys())
    arr_name = [n for n in ds.variables.keys() if ds[n].ndim==3][0]
    
    ncfile = nc.Dataset(path_color_interp , mode='w', format='NETCDF4_CLASSIC') 
    
    time_dim = ncfile.createDimension('time', None) # unlimited axis (can be appended to).
    lon_dim = ncfile.createDimension(var_names[1], ds[var_names[1]].shape[0])
    lat_dim = ncfile.createDimension(var_names[2], ds[var_names[2]].shape[0])
    
    # Define two variables with the same names as dimensions,
    # a conventional way to define "coordinate variables".
    time = ncfile.createVariable(var_names[0], np.float64, (var_names[0],))
    time.units = 'day as %Y%m%d.%f'
    time.long_name = var_names[0]

    lon = ncfile.createVariable(var_names[1], np.float32, (var_names[1],))
    lon.units = 'degrees_east'
    lon.long_name = var_names[1]

    lat = ncfile.createVariable(var_names[2], np.float32, (var_names[2],))
    lat.units = 'degrees_north'
    lat.long_name = var_names[2]

    # Define a 3D variable to hold the data
    data = ncfile.createVariable(var_names[3],np.float32,
                                 (var_names[0],var_names[2],var_names[1])) 
    # note: unlimited dimension is leftmost
    data.units = 'W m-2'
    data.standard_name = arr_name
    
    # fill in data
    time[:] = ds[var_names[0]][:]
    lon[:] = ds[var_names[1]][:]
    lat[:] = ds[var_names[2]][:]
    data[:] = arr_interp

    ncfile.close(); 
    logging.info('Saved netCDF Dataset with interpolation!')

In [17]:
def interpolate_nans(arr_color):
    
    if ~np.all(np.isnan(arr_color)):
        
        idxs_to_interp = np.where(np.isnan(arr_color))
        idxs_interp_from = np.where(~np.isnan(arr_color))
        
        try:
            arr_color[idxs_to_interp] = griddata(idxs_interp_from, 
                                             arr_color[idxs_interp_from], 
                                             idxs_to_interp)
        except QhullError:
            logging.warning('not interpulating - QHull error')
            
    return arr_color

In [18]:
def save_interpolated_nans_temporal(arr_color_ref, ds_color, path_color_interp):
    
    arr_interp = np.full(arr_color_ref.shape, np.nan, dtype=np.float16)
    
    for i in range(arr_color_ref.shape[1]):
        for j in range(arr_color_ref.shape[2]):
            arr_color = load_arr_2D(arr_color_ref, np.index_exp[:,i,j])
            arr_color = interpolate_nans(arr_color.astype(np.float16))
        
            arr_interp[:,i,j] = arr_color
            
    save_nc_with_interp(arr_interp, ds_arr_color, path_color_interp)

### Make plotter

In [33]:
def create_plotter(output_path, frame_rate):
    p = pv.Plotter(notebook=False, off_screen=True,
                  window_size=[1920 * 2, 1080 * 2], 
                   multi_samples=16, lighting='three lights')
    p.open_movie(output_path, framerate=frame_rate)
    p.set_background('black')
    
    return p

In [20]:
def add_BG(p, bg):
    floor_thickness = 40
    p.add_mesh(pv.Cube(center=(bg.center[0], bg.center[1], -floor_thickness), 
                       x_length=bg.bounds[1]-bg.bounds[0],
                       y_length=bg.bounds[3]-bg.bounds[2], 
                       z_length=floor_thickness*2-0.02),
               color='white')

    p.add_mesh(bg, rgb=True, scalars='PNGImage')
    
    return p

### Make mesh

In [21]:
def make_floor_vertices(arr, cube_size):

    xs,ys = np.meshgrid(range(arr.shape[0]),range(arr.shape[1]))
    
    vertices_base = np.vstack((xs.T.flatten()*cube_size, 
                        ys.T.flatten()* cube_size,
                        np.zeros(xs.size))).T
    
    xs,ys = np.meshgrid(range(arr.shape[0]),[arr.shape[1]])
    vertices_edge_y = np.vstack((xs.T.flatten()*cube_size, 
                        ys.T.flatten()*cube_size,
                        np.zeros(xs.size))).T
    
    xs,ys = np.meshgrid(arr.shape[0],range(arr.shape[1]))
    vertices_edge_x = np.vstack((xs.T.flatten()* cube_size, 
                        ys.T.flatten()* cube_size,
                        np.zeros(xs.size))).T
        
    return vertices_base, vertices_edge_y, vertices_edge_x

In [22]:
def concat_all_vertices(arr, vertices_base, vertices_edge_y, vertices_edge_x, cube_size=1):
    
    vertices_ceil = vertices_base.copy()
    vertices_ceil[:,2] = (arr.flatten()/10)**2-(np.nanmin(arr.flatten()/10)**2)
    
    vertices = np.concatenate((vertices_base,
                               vertices_edge_y,
                               vertices_edge_x,
                               vertices_ceil,
                               vertices_ceil+[0,1*cube_size,0],
                               vertices_ceil+[1*cube_size,1*cube_size,0],
                               vertices_ceil+[1*cube_size,0,0],
                          ))
    
    return vertices

In [23]:
def make_vertices(arr, cube_size=1):
    vertices_base, vertices_edge_y, vertices_edge_x = make_floor_vertices(arr, cube_size)
    vertices = concat_all_vertices(arr, vertices_base, vertices_edge_y, vertices_edge_x, cube_size)
    
    n_floor_v = vertices_base.shape[0]+vertices_edge_y.shape[0]+vertices_edge_x.shape[0]
    return vertices, n_floor_v

In [24]:
def make_floor_faces(arr):
    faces = []
    it = np.nditer(arr, flags=['c_index','multi_index'])

    for x in it:
        if not np.isnan(x):
            
            neigh_right_idx = it.index+1 if it.multi_index[1]!=arr.shape[1]-1 \
                                else arr.size + it.multi_index[0] 
            neigh_down_idx = it.index+arr.shape[1] if it.multi_index[0]!=arr.shape[0]-1 \
                                else arr.size + arr.shape[0] + it.multi_index[1]
            
            neigh_diag_idx = it.index+arr.shape[1]+1
            if neigh_down_idx>arr.size:
                neigh_diag_idx = neigh_down_idx+1
            elif neigh_right_idx>arr.size-1:
                neigh_diag_idx = neigh_right_idx+1

            faces.append(np.array([4,
                                   it.index,
                                   neigh_right_idx,
                                   neigh_diag_idx,
                                   neigh_down_idx,
                                  ]))
            
    return faces

In [25]:
def make_ceil_faces(arr, n_v_floor):

    faces = []
    it = np.nditer(arr, flags=['c_index','multi_index'])

    for x in it:
        if not np.isnan(x):
            faces.append(np.array([4,
                                   n_v_floor+it.index,
                                   n_v_floor+arr.size+it.index,
                                   n_v_floor+arr.size*2+it.index,
                                   n_v_floor+arr.size*3+it.index,
                                  ]))
    
    return faces

In [26]:
def make_side_faces(faces):
    side_faces = []

    for i in range(len(faces)//2):
        side_faces.append(np.array([4,
                               faces[i][1],
                               faces[i][2],
                               faces[len(faces)//2+i][2],
                               faces[len(faces)//2+i][1],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][1],
                               faces[i][4],
                               faces[len(faces)//2+i][4],
                               faces[len(faces)//2+i][1],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][2],
                               faces[i][3],
                               faces[len(faces)//2+i][3],
                               faces[len(faces)//2+i][2],
                              ]))
        side_faces.append(np.array([4,
                               faces[i][3],
                               faces[i][4],
                               faces[len(faces)//2+i][4],
                               faces[len(faces)//2+i][3],
                              ]))
        
    return side_faces

In [27]:
def make_faces(arr, n_floor_v):
    faces = make_floor_faces(arr)
    faces.extend(make_ceil_faces(arr, n_floor_v))
    faces.extend(make_side_faces(faces))
    
    return faces

In [28]:
def make_face_color(arr_color_flat, arr_mesh, clim):
    arr_color_flat = np.delete(arr_color_flat, np.isnan(arr_mesh.flatten(order='F')))
    
    faces_color = np.concatenate((np.tile(arr_color_flat, 2),
                               np.tile(arr_color_flat, (4,1)).flatten(order='F')))
    
#     arr_color_opacity = (np.abs(arr_color_flat-np.mean([clim[0],clim[1]])) / (np.ptp(clim)/2))*3
    
#     faces_opacity = np.concatenate((np.tile(arr_color_opacity, 2),
#                                np.tile(arr_color_opacity, (4,1)).flatten(order='F')))    
    
#     faces_opacity[faces_opacity>1] = 1.0
    
#     faces_opacity = faces_opacity*0.8
    
    return faces_color

In [29]:
def make_surface(arr_mesh, cube_size, arr_color, clim):
    
    vertices, n_floor_v = make_vertices(arr_mesh, cube_size)
    faces = make_faces(arr_mesh, n_floor_v)
    surf = pv.PolyData(vertices, faces)
    faces_color = make_face_color(arr_color.flatten(order='F'), arr_mesh, clim)
    surf["colors"] = faces_color
    
    return surf

### Event

In [30]:
def get_events(event_csv_path, time_arr, data_shape, lat_min, lat_max, long_min, long_max):
    """ Gets a csv file path of highlighted events that should be annotated in the rendering,
    (as a text bubble on a specified map location) 
    it converts the data to x,y mesh position and timeframe indexing and returns it as a dictionary.

    Args:
        event_csv_path: str path to csv file. Each row in the csv is an event. csv should have 5 columns:
            "first_date" - first date of event in the format e.g. 20001231
            "last_date" - last date of event
            "longitude" - event longitude coordinate
            "latitude" - event latitude coordinate
            "text" - event text
        time_arr: 1D array. Part of the netCDF dataset
        long_arr: 1D array. Part of the netCDF dataset
        lat_arr: 1D array. Part of the netCDF dataset

    Returns:
        event_dict - dict representation of the csv where each key is a timeframe index.
                example: {"0":[[-20,250,150],"event text example"]}
                means that there is only one event, it would be displayed at the first (0) timeframe,
                at location [x,y,z] of the mesh coordinates.

    Note:
        for now also processing timepoints outside of user input timepoints.
    TODO:
        Need to check that the csv format is correct.
        for now not checking that long and lat values are close to values in arrays:
        e.g. [abs(long_arr[idx_long[i]]-val)<max_dist for i,val in enumerate(loc_arr[0])]
    """

    lat_arr = np.linspace(lat_min, lat_max, data_shape[0])
    long_arr = np.linspace(long_min, long_max, data_shape[1])
        
    event_dict = {}

    df = pd.read_csv(event_csv_path, dtype={"first_date": str, "last_date": str})
    for i in df.index:
        try:
            idx_first_date = time_arr.index(df.loc[i, "first_date"])
            idx_last_date = time_arr.index(df.loc[i, "last_date"])
        except Exception as e:
            logging.exception(e)
            logging.warning(f'event csv has an invalid date - row #{i}')
            continue
            
        point = [(np.abs(long_arr - df.loc[i, "longitude"])).argmin(),
                 (np.abs(lat_arr - df.loc[i, "latitude"])).argmin(),
                 150]  # x,y,z
        
        if (not lat_min<point[0]<lat_max) or (not long_min<point[0]<long_max):
            logging.warning(f'event csv has an invalid location - row #{i}')
            continue
                                    
        text = (df.loc[i, "text"]).replace('\\n', '\n')
        for idx in range(idx_first_date, idx_last_date + 1):

            if str(idx) not in event_dict:
                event_dict[str(idx)] = [[point], [text]]
            else:
                event_dict[str(idx)][0].append(point)
                event_dict[str(idx)][1].append(text)

    return event_dict

In [31]:
def remove_actors(p):
    p.remove_actor('mesh')
    p.remove_actor('date')
    
    return p

### Video

In [32]:
ds_mesh, arr_mesh_ref = load_nc(path_input_mesh)
ds_color, arr_color_ref = load_nc(path_input_color)

if is_interp=='temporal':
    path_color_interp = f'{os.path.splitext(path_input_color)}_temporalInterpolation.nc'
    if not os.path.exists(path_color_interp):
        save_nc_with_interp(arr_color_ref, ds_color, path_color_interp)
    ds_color, arr_color_ref = load_nc(path_color_interp)

dates_mesh = format_time_arr_yearmonthday(ds_mesh)
dates_color = format_time_arr_yearmonthday(ds_color)
mesh_dates_idxs_in_color = map_mesh_dates_to_color_idx(dates_mesh, dates_color)

lat_min, lat_max = find_min_max(ds_mesh["latitude"][:], ds_color["latitude"][:], lat_range)
long_min, long_max = find_min_max(ds_mesh["longitude"][:], ds_color["longitude"][:], long_range)

bg, data_shape = make_BG_mesh(path_input_BG, bg_padding, 
                              lat_min, lat_max, long_min, long_max)

events_per_time_idx_dict = get_events(event_csv_path, time_arr_mesh, data_shape,
                                      lat_min, lat_max, long_min, long_max,
                                     ) if event_csv_path is not None else {}

p = create_plotter(output_path, frame_rate)
p = add_BG(p, bg)

for i_date in range(n_timepoints-1):
    
    arr_mesh_curr = load_and_resize_arr(arr_mesh_ref, i_date, ds_mesh, True, 
                                        (lat_min, lat_max), (long_min, long_max), 
                                        data_shape, cube_size)
    arr_mesh_next = load_and_resize_arr(arr_mesh_ref, i_date+1, ds_mesh, True, 
                                        (lat_min, lat_max), (long_min, long_max), 
                                        data_shape, cube_size)
    
    arr_color_curr = load_and_resize_arr(arr_color_ref, mesh_dates_idxs_in_color[i_date], 
                                   ds_color, False,
                                  (lat_min, lat_max), (long_min, long_max),
                                  data_shape, cube_size, is_interp=='spatial')
    arr_color_next = load_and_resize_arr(arr_color_ref, mesh_dates_idxs_in_color[i_date+1], 
                                   ds_color, False,
                                  (lat_min, lat_max), (long_min, long_max),
                                  data_shape, cube_size, is_interp=='spatial')
    
    for i_interp in range(n_interp+1):
        
        p = remove_actors(p)
        
        arr_mesh = interpolate_between_slices(arr_mesh_curr, arr_mesh_next, i_interp/n_interp)
        arr_color = interpolate_between_slices(arr_color_curr, arr_color_next, i_interp/n_interp)
        
        surf = make_surface(arr_mesh, cube_size, arr_color, clim)

        p.add_mesh(surf, scalars="colors", cmap=cmap, clim=clim, show_edges=True, lighting=True, name='mesh')
        if str(i_date) in events_per_time_idx_dict:
            p.add_point_labels(events_per_time_idx_dict[str(i_date)][0],
                               events_per_time_idx_dict[str(i_date)][1],
                               name='event_text', italic=True, font_size=40,
                               shape_color='black', point_color='pink', point_size=40,
                               shape_opacity=0.5,
                               render_points_as_spheres=True, always_visible=True, shadow=True)
            
        if i_date==0 and i_interp==0:
            p.camera_position = 'xy'
            #p.camera.position = (p.camera_position.position[0],p.camera_position.position[1],p.camera_position.position[2]+2000)
            p.camera.zoom(2)
            p.camera.azimuth -= 30
            p.camera.elevation = -20
        
        p.camera.azimuth += 0.02
        p.write_frame()

pv.close_all()

INFO:root:Read netCDF surface_upward_sensible_heat_flux data. Shape=(460, 601, 1233)
INFO:root:Read netCDF LAI data. Shape=(144, 602, 1293)


True

In [9]:
import ffmpeg
ffmpeg.input(output_path).output('out1.mp4', vcodec='libx265', preset='fast', crf=32).run()

# tags explained
#https://gist.github.com/tayvano/6e2d456a9897f55025e25035478a3a50:


ffmpeg version 4.4.2 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11.3.0 (conda-forge gcc 11.3.0-19)
  configuration: --prefix=/home/conda/feedstock_root/build_artifacts/ffmpeg_1671040255947/_h_env_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_placehold_plac --cc=/home/conda/feedstock_root/build_artifacts/ffmpeg_1671040255947/_build_env/bin/x86_64-conda-linux-gnu-cc --cxx=/home/conda/feedstock_root/build_artifacts/ffmpeg_1671040255947/_build_env/bin/x86_64-conda-linux-gnu-c++ --nm=/home/conda/feedstock_root/build_artifacts/ffmpeg_1671040255947/_build_env/bin/x86_64-conda-linux-gnu-nm --ar=/home/conda/feedstock_root/build_artifacts/ffmpeg_1671040255947/_build_env/bin/x86_64-conda-linux-gnu-ar --disable-doc --disable-openssl --enable-avresample --enable-demuxer=dash --enable-hardcoded-tables --enable-libfreetype --enable-libfontconfig --enable-libo

(None, None)